In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from textblob import TextBlob
import nltk
import tslearn
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import preprocessing
import timestring
import datetime

In [2]:
missing_values = ["n/a", "na", "--"]

In [66]:
stock = pd.read_csv("DataApril.csv", sep = ",", na_values = missing_values)
gold = pd.read_csv("GoldDataFinal.csv", sep = ",",na_values = missing_values)
oil = pd.read_csv("OilDataFinal.csv", sep = ",",na_values = missing_values)
news = pd.read_csv("newsdata1.csv", sep=",",na_values = missing_values)
stock.head()



,Id,stockdate,sector,subsector,companyname,stockcode,openprice,highprice,lowprice,lastprice,volume,created_at
0,821,4/17/2019,main_market,healthcare,ADVENTA,7191,0.35,0.355,0.35,0.35,49700,4/19/2019 17:35
1,2684,4/17/2019,main_market,healthcare,DPHARMA,7148,1.31,1.310,1.30,1.31,902500,4/19/2019 17:35
2,3688,4/17/2019,main_market,healthcare,HARTA,5168,4.69,4.760,4.69,4.73,6626400,4/19/2019 17:35
3,4692,4/17/2019,main_market,healthcare,IHH,5225,5.58,5.580,5.41,5.48,4107000,4/19/2019 17:35
4,5696,4/17/2019,main_market,healthcare,KOSSAN,7153,3.54,3.550,3.52,3.54,555400,4/19/2019 17:35


In [67]:
stock_missing = stock.isnull().sum()
gold_missing = gold.isnull().sum()
silver_missing = silver.isnull().sum()
oil_missing = oil.isnull().sum()
gas_missing = gas.isnull().sum()
news_missing = news.isnull().sum()

In [68]:
news["newsdate"]= news["newsdate"].apply(lambda x:timestring.Date(x).date)
news.head()

,newsdate,companyname,headline,article,link
0,2019-05-13 08:57:00,CIMB,UOA REIT in acquisition mode,KUALA LUMPUR: Almost a year after the RM120 mi...,https://www.theedgemarkets.com/article/uoa-rei...
1,2019-05-10 19:07:00,CIMB,CIMB Bank lowers base lending rate following O...,KUALA LUMPUR (May 10): CIMB Bank Bhd and CIMB ...,https://www.theedgemarkets.com/article/cimb-ba...
2,2019-05-10 08:22:00,CIMB,"Axiata 1Q19F earnings to rebound, says CIMB Re...",KUALA LUMPUR (May 10): CIMB IB Research has ma...,https://www.theedgemarkets.com/article/axiata-...
3,2019-05-09 11:08:00,CIMB,Minimal impact from 25bps OPR cut on banking s...,Banking sector\nMaintain overweight\nBank Nega...,https://www.theedgemarkets.com/article/minimal...
4,2019-05-09 08:20:00,CIMB,"CIMB Research downgrades Cypark, lowers target...",KUALA LUMPUR (May 9): CIMB IB Research has dow...,https://www.theedgemarkets.com/article/cimb-re...


In [49]:
news["newsdate"] = news["newsdate"].apply(lambda x:pd.datetime.date(pd.to_datetime(x)))
news.head()

,newsdate,companyname,headline,article,link
0,2019-05-13,CIMB,UOA REIT in acquisition mode,KUALA LUMPUR: Almost a year after the RM120 mi...,https://www.theedgemarkets.com/article/uoa-rei...
1,2019-05-10,CIMB,CIMB Bank lowers base lending rate following O...,KUALA LUMPUR (May 10): CIMB Bank Bhd and CIMB ...,https://www.theedgemarkets.com/article/cimb-ba...
2,2019-05-10,CIMB,"Axiata 1Q19F earnings to rebound, says CIMB Re...",KUALA LUMPUR (May 10): CIMB IB Research has ma...,https://www.theedgemarkets.com/article/axiata-...
3,2019-05-09,CIMB,Minimal impact from 25bps OPR cut on banking s...,Banking sector\nMaintain overweight\nBank Nega...,https://www.theedgemarkets.com/article/minimal...
4,2019-05-09,CIMB,"CIMB Research downgrades Cypark, lowers target...",KUALA LUMPUR (May 9): CIMB IB Research has dow...,https://www.theedgemarkets.com/article/cimb-re...


In [50]:
news["headlinepolarity"] = [TextBlob(news["headline"].iloc[i]).polarity for i in range(0,news.shape[0])]

In [51]:
news["articlepolarity"] = [TextBlob(news["article"].iloc[i]).polarity for i in range(0,news.shape[0])]
news.head()

,newsdate,companyname,headline,article,link,headlinepolarity,articlepolarity
0,2019-05-13,CIMB,UOA REIT in acquisition mode,KUALA LUMPUR: Almost a year after the RM120 mi...,https://www.theedgemarkets.com/article/uoa-rei...,0.0,0.137771
1,2019-05-10,CIMB,CIMB Bank lowers base lending rate following O...,KUALA LUMPUR (May 10): CIMB Bank Bhd and CIMB ...,https://www.theedgemarkets.com/article/cimb-ba...,-0.4,-0.034259
2,2019-05-10,CIMB,"Axiata 1Q19F earnings to rebound, says CIMB Re...",KUALA LUMPUR (May 10): CIMB IB Research has ma...,https://www.theedgemarkets.com/article/axiata-...,0.0,-0.118750
3,2019-05-09,CIMB,Minimal impact from 25bps OPR cut on banking s...,Banking sector\nMaintain overweight\nBank Nega...,https://www.theedgemarkets.com/article/minimal...,-0.1,-0.015848
4,2019-05-09,CIMB,"CIMB Research downgrades Cypark, lowers target...",KUALA LUMPUR (May 9): CIMB IB Research has dow...,https://www.theedgemarkets.com/article/cimb-re...,0.0,-0.033333


In [52]:
news['polarity'] = news[['headlinepolarity','articlepolarity']].sum(axis=1)
news.head()

,newsdate,companyname,headline,article,link,headlinepolarity,articlepolarity,polarity
0,2019-05-13,CIMB,UOA REIT in acquisition mode,KUALA LUMPUR: Almost a year after the RM120 mi...,https://www.theedgemarkets.com/article/uoa-rei...,0.0,0.137771,0.137771
1,2019-05-10,CIMB,CIMB Bank lowers base lending rate following O...,KUALA LUMPUR (May 10): CIMB Bank Bhd and CIMB ...,https://www.theedgemarkets.com/article/cimb-ba...,-0.4,-0.034259,-0.434259
2,2019-05-10,CIMB,"Axiata 1Q19F earnings to rebound, says CIMB Re...",KUALA LUMPUR (May 10): CIMB IB Research has ma...,https://www.theedgemarkets.com/article/axiata-...,0.0,-0.118750,-0.118750
3,2019-05-09,CIMB,Minimal impact from 25bps OPR cut on banking s...,Banking sector\nMaintain overweight\nBank Nega...,https://www.theedgemarkets.com/article/minimal...,-0.1,-0.015848,-0.115848
4,2019-05-09,CIMB,"CIMB Research downgrades Cypark, lowers target...",KUALA LUMPUR (May 9): CIMB IB Research has dow...,https://www.theedgemarkets.com/article/cimb-re...,0.0,-0.033333,-0.033333


In [53]:
RHBBANK_news = news
CIMB_news = news

In [55]:
#RHBBANK_news = RHBBANK_news.iloc[np.where(news["companyname"]=="RHBBANK")]
CIMB_news = CIMB_news.iloc[np.where(news["companyname"]=="CIMB")]
#RHBBANK_news
CIMB_news

,newsdate,companyname,headline,article,link,headlinepolarity,articlepolarity,polarity
0,2019-05-13,CIMB,UOA REIT in acquisition mode,KUALA LUMPUR: Almost a year after the RM120 mi...,https://www.theedgemarkets.com/article/uoa-rei...,0.000000,0.137771,0.137771
1,2019-05-10,CIMB,CIMB Bank lowers base lending rate following O...,KUALA LUMPUR (May 10): CIMB Bank Bhd and CIMB ...,https://www.theedgemarkets.com/article/cimb-ba...,-0.400000,-0.034259,-0.434259
2,2019-05-10,CIMB,"Axiata 1Q19F earnings to rebound, says CIMB Re...",KUALA LUMPUR (May 10): CIMB IB Research has ma...,https://www.theedgemarkets.com/article/axiata-...,0.000000,-0.118750,-0.118750
3,2019-05-09,CIMB,Minimal impact from 25bps OPR cut on banking s...,Banking sector\nMaintain overweight\nBank Nega...,https://www.theedgemarkets.com/article/minimal...,-0.100000,-0.015848,-0.115848
4,2019-05-09,CIMB,"CIMB Research downgrades Cypark, lowers target...",KUALA LUMPUR (May 9): CIMB IB Research has dow...,https://www.theedgemarkets.com/article/cimb-re...,0.000000,-0.033333,-0.033333
5,2019-05-08,CIMB,Malaysian palm stocks seen near 3 mil tonnes u...,KUALA LUMPUR (May 8): Malaysian palm stocks ar...,https://www.theedgemarkets.com/article/malaysi...,0.100000,0.053921,0.153921
6,2019-05-08,CIMB,Banks lobbying MoF for provisions to be fully ...,THE banking industry is lobbying the Ministry ...,https://www.theedgemarkets.com/article/banks-l...,0.000000,0.059209,0.059209
7,2019-05-08,CIMB,"Long CIMB, short Public Bank as value, growth ...","SINGAPORE (May 8): There is a ""strong case"" fo...",https://www.theedgemarkets.com/article/long-ci...,-0.016667,0.091667,0.075000
8,2019-05-08,CIMB,Hartalega down after reporting weaker 4Q results,KUALA LUMPUR (May 8): Hartalega Holdings Bhd s...,https://www.theedgemarkets.com/article/hartale...,-0.155556,0.106250,-0.049306
9,2019-05-07,CIMB,Former Bank of Thailand deputy governor joins ...,"KUALA LUMPUR (May 7): Tongurai Limpiti, the fo...",https://www.theedgemarkets.com/article/former-...,0.000000,0.000000,0.000000


In [56]:
# RHBBANK_news.index = pd.to_datetime(RHBBANK_news["newsdate"])
# RHBBANK_news.loc[:'2019-04-01']
CIMB_news.index = pd.to_datetime(CIMB_news["newsdate"])
CIMB_news.loc[:'2019-03-01']

,newsdate,companyname,headline,article,link,headlinepolarity,articlepolarity,polarity
newsdate,,,,,,,,
2019-05-13,2019-05-13,CIMB,UOA REIT in acquisition mode,KUALA LUMPUR: Almost a year after the RM120 mi...,https://www.theedgemarkets.com/article/uoa-rei...,0.000000,0.137771,0.137771
2019-05-10,2019-05-10,CIMB,CIMB Bank lowers base lending rate following O...,KUALA LUMPUR (May 10): CIMB Bank Bhd and CIMB ...,https://www.theedgemarkets.com/article/cimb-ba...,-0.400000,-0.034259,-0.434259
2019-05-10,2019-05-10,CIMB,"Axiata 1Q19F earnings to rebound, says CIMB Re...",KUALA LUMPUR (May 10): CIMB IB Research has ma...,https://www.theedgemarkets.com/article/axiata-...,0.000000,-0.118750,-0.118750
2019-05-09,2019-05-09,CIMB,Minimal impact from 25bps OPR cut on banking s...,Banking sector\nMaintain overweight\nBank Nega...,https://www.theedgemarkets.com/article/minimal...,-0.100000,-0.015848,-0.115848
2019-05-09,2019-05-09,CIMB,"CIMB Research downgrades Cypark, lowers target...",KUALA LUMPUR (May 9): CIMB IB Research has dow...,https://www.theedgemarkets.com/article/cimb-re...,0.000000,-0.033333,-0.033333
2019-05-08,2019-05-08,CIMB,Malaysian palm stocks seen near 3 mil tonnes u...,KUALA LUMPUR (May 8): Malaysian palm stocks ar...,https://www.theedgemarkets.com/article/malaysi...,0.100000,0.053921,0.153921
2019-05-08,2019-05-08,CIMB,Banks lobbying MoF for provisions to be fully ...,THE banking industry is lobbying the Ministry ...,https://www.theedgemarkets.com/article/banks-l...,0.000000,0.059209,0.059209
2019-05-08,2019-05-08,CIMB,"Long CIMB, short Public Bank as value, growth ...","SINGAPORE (May 8): There is a ""strong case"" fo...",https://www.theedgemarkets.com/article/long-ci...,-0.016667,0.091667,0.075000
2019-05-08,2019-05-08,CIMB,Hartalega down after reporting weaker 4Q results,KUALA LUMPUR (May 8): Hartalega Holdings Bhd s...,https://www.theedgemarkets.com/article/hartale...,-0.155556,0.106250,-0.049306


In [57]:
# RHBBANK_news1 = RHBBANK_news.loc["2019-04-30":"2019-04-01"].resample("B").sum()
# RHBBANK_news1
CIMB_news1 = CIMB_news.loc["2019-05-13":"2019-03-01"].resample("B").sum()
CIMB_news1

,headlinepolarity,articlepolarity,polarity
newsdate,,,
2019-03-13,0.000000,0.024766,0.024766
2019-03-14,0.000000,0.000000,0.000000
2019-03-15,0.160000,0.221097,0.381097
2019-03-18,0.000000,0.194209,0.194209
2019-03-19,0.053333,0.095898,0.149231
2019-03-20,0.000000,0.114464,0.114464
2019-03-21,0.500000,0.442781,0.942781
2019-03-22,0.318182,0.172218,0.490400
2019-03-25,0.100000,0.146245,0.246245


In [69]:
# RHBBANK_stock = stock.loc[np.where(stock["companyname"]=="RHBBANK")]
# RHBBANK_stock
CIMB_stock = stock.loc[np.where(stock["companyname"]=="CIMB")]
CIMB_stock


,Id,stockdate,sector,subsector,companyname,stockcode,openprice,highprice,lowprice,lastprice,volume,created_at
220,223009,4/17/2019,main_market,finance,CIMB,1023,5.01,5.08,5.00,5.05,12978800,4/19/2019 17:38
1409,223008,4/16/2019,main_market,finance,CIMB,1023,5.06,5.07,5.01,5.05,6761600,4/19/2019 17:38
2559,223007,4/15/2019,main_market,finance,CIMB,1023,5.08,5.10,5.05,5.05,5921900,4/19/2019 17:38
3726,223006,4/14/2019,main_market,finance,CIMB,1023,5.07,5.13,5.06,5.06,3695800,4/19/2019 17:38
4950,223005,4/11/2019,main_market,finance,CIMB,1023,5.07,5.08,5.06,5.06,3507200,4/19/2019 17:38
6138,223004,4/10/2019,main_market,finance,CIMB,1023,5.08,5.12,5.06,5.06,7562100,4/19/2019 17:38
7354,223003,4/9/2019,main_market,finance,CIMB,1023,5.09,5.10,5.06,5.08,3525600,4/19/2019 17:38
8531,223002,4/8/2019,main_market,finance,CIMB,1023,5.10,5.11,5.07,5.09,12074500,4/19/2019 17:38
9739,223001,4/7/2019,main_market,finance,CIMB,1023,5.10,5.12,5.08,5.09,7378400,4/19/2019 17:38
10935,223000,4/4/2019,main_market,finance,CIMB,1023,5.10,5.13,5.10,5.11,2330800,4/19/2019 17:38


In [70]:
# RHBBANK_stock["stockdate"] = RHBBANK_stock["stockdate"].apply(lambda x:pd.datetime.date(pd.to_datetime(x)))
# RHBBANK_stock.head()
CIMB_stock["stockdate"] = CIMB_stock["stockdate"].apply(lambda x:pd.datetime.date(pd.to_datetime(x)))
CIMB_stock.head()

,Id,stockdate,sector,subsector,companyname,stockcode,openprice,highprice,lowprice,lastprice,volume,created_at
220,223009,2019-04-17,main_market,finance,CIMB,1023,5.01,5.08,5.00,5.05,12978800,4/19/2019 17:38
1409,223008,2019-04-16,main_market,finance,CIMB,1023,5.06,5.07,5.01,5.05,6761600,4/19/2019 17:38
2559,223007,2019-04-15,main_market,finance,CIMB,1023,5.08,5.10,5.05,5.05,5921900,4/19/2019 17:38
3726,223006,2019-04-14,main_market,finance,CIMB,1023,5.07,5.13,5.06,5.06,3695800,4/19/2019 17:38
4950,223005,2019-04-11,main_market,finance,CIMB,1023,5.07,5.08,5.06,5.06,3507200,4/19/2019 17:38


In [60]:
gold["golddate"] = gold["golddate"].apply(lambda x:pd.datetime.date(pd.to_datetime(x)))
gold.head()

,Id,golddate,openprice,highprice,lowprice,closeprice,volume,created_at
0,1,2015-03-12,15.560,15.565,15.465,15.493,94,4/19/2019 17:54
1,2,2015-03-13,15.610,15.620,15.475,15.477,59,4/19/2019 17:54
2,3,2015-03-15,15.627,15.627,15.565,15.585,0,4/19/2019 17:54
3,4,2015-03-16,15.690,15.715,15.500,15.601,71,4/19/2019 17:54
4,5,2015-03-17,15.490,15.635,15.490,15.562,39,4/19/2019 17:54


In [62]:
oil["oildate"] = oil["oildate"].apply(lambda x:pd.datetime.date(pd.to_datetime(x)))
oil.head()

,Id,oildate,openprice,highprice,lowprice,closeprice,volume,created_at
0,1,2015-03-12,57.919998,58.810001,56.509998,57.080002,129734,4/19/2019 17:54
1,2,2015-03-13,57.029999,57.400002,54.430000,54.669998,84085,4/19/2019 17:54
2,3,2015-03-16,54.410000,54.639999,52.500000,53.439999,15638,4/19/2019 17:54
3,4,2015-03-17,54.099998,54.380001,52.570000,53.509998,240285,4/19/2019 17:54
4,5,2015-03-18,53.279999,56.840000,52.680000,55.910000,262440,4/19/2019 17:54


In [77]:
# RHBBANK_stock.index = pd.to_datetime(RHBBANK_stock["stockdate"])
# RHBBANK_stock.loc['2019-04-01':]
CIMB_stock.index = pd.to_datetime(CIMB_stock["stockdate"])
CIMB_stock.loc['2019-05-17':]

,Id,stockdate,sector,subsector,companyname,stockcode,openprice,highprice,lowprice,lastprice,volume,created_at
stockdate,,,,,,,,,,,,
2019-04-17,223009,2019-04-17,main_market,finance,CIMB,1023,5.01,5.08,5.00,5.05,12978800,4/19/2019 17:38
2019-04-16,223008,2019-04-16,main_market,finance,CIMB,1023,5.06,5.07,5.01,5.05,6761600,4/19/2019 17:38
2019-04-15,223007,2019-04-15,main_market,finance,CIMB,1023,5.08,5.10,5.05,5.05,5921900,4/19/2019 17:38
2019-04-14,223006,2019-04-14,main_market,finance,CIMB,1023,5.07,5.13,5.06,5.06,3695800,4/19/2019 17:38
2019-04-11,223005,2019-04-11,main_market,finance,CIMB,1023,5.07,5.08,5.06,5.06,3507200,4/19/2019 17:38
2019-04-10,223004,2019-04-10,main_market,finance,CIMB,1023,5.08,5.12,5.06,5.06,7562100,4/19/2019 17:38
2019-04-09,223003,2019-04-09,main_market,finance,CIMB,1023,5.09,5.10,5.06,5.08,3525600,4/19/2019 17:38
2019-04-08,223002,2019-04-08,main_market,finance,CIMB,1023,5.10,5.11,5.07,5.09,12074500,4/19/2019 17:38
2019-04-07,223001,2019-04-07,main_market,finance,CIMB,1023,5.10,5.12,5.08,5.09,7378400,4/19/2019 17:38


In [84]:
gold.index = pd.to_datetime(gold["golddate"])
gold1 = gold.loc['2019-03-01':]

In [85]:
oil.index = pd.to_datetime(oil["oildate"])
oil1 = oil.loc['2019-03-01':]

In [20]:
RHBBANK = pd.concat([RHBBANK_stock["lastprice"]
                      ,RHBBANK_news1["polarity"]]
                      , keys = ["stockprice","newspolarity"], axis = 1).ffill().bfill()
RHBBANK

,stockprice,newspolarity
2019-04-01,5.65,0.557766
2019-04-02,5.75,0.557766
2019-04-03,5.75,0.000000
2019-04-04,5.73,0.000000
2019-04-05,5.73,-0.141253
2019-04-07,5.68,-0.141253
2019-04-08,5.66,0.840428
2019-04-09,5.69,0.287677
2019-04-10,5.68,0.000000
2019-04-11,5.75,0.275617


In [88]:
CIMB = pd.concat([CIMB_stock["lastprice"]
                      ,CIMB_news1["polarity"], gold1["closeprice"], oil1["closeprice"]]
                      , keys = ["stockprice","newspolarity", "goldprice", "oilprice"], axis = 1).ffill().bfill()
CIMB

,stockprice,newspolarity,goldprice,oilprice
2019-03-01,5.64,0.024766,15.256,65.070000
2019-03-03,5.64,0.024766,15.160,65.070000
2019-03-04,5.59,0.024766,15.105,65.669998
2019-03-05,5.52,0.024766,15.105,65.860001
2019-03-06,5.45,0.024766,15.085,65.989998
2019-03-07,5.39,0.024766,15.040,66.300003
2019-03-08,5.39,0.024766,15.349,65.739998
2019-03-10,5.28,0.024766,15.338,65.739998
2019-03-11,5.37,0.024766,15.274,66.580002
2019-03-12,5.45,0.024766,15.413,66.669998


In [90]:
CIMB.to_csv("CIMB2.csv")

In [82]:
CIMB_news1
# gold

,headlinepolarity,articlepolarity,polarity
newsdate,,,
2019-03-13,0.000000,0.024766,0.024766
2019-03-14,0.000000,0.000000,0.000000
2019-03-15,0.160000,0.221097,0.381097
2019-03-18,0.000000,0.194209,0.194209
2019-03-19,0.053333,0.095898,0.149231
2019-03-20,0.000000,0.114464,0.114464
2019-03-21,0.500000,0.442781,0.942781
2019-03-22,0.318182,0.172218,0.490400
2019-03-25,0.100000,0.146245,0.246245


In [29]:
CIMB.to_csv('CIMB2.csv')

In [30]:
RHBBANK.to_csv('RHBBANK.csv')